In [1]:
import numpy as np
import keras
from keras import applications
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras import optimizers
import keras.backend as K
from keras.models import load_model

In [2]:
x_train=np.load('dataset/x_train_np.npy')
x_cv=np.load('dataset/x_cv_np.npy')
y_train=np.load('dataset/y_train_np.npy')
y_cv=np.load('dataset/y_cv_np.npy')

In [3]:
x_test=np.load('dataset/x_test_np.npy', allow_pickle=True)
y_test=np.load('dataset/y_test_np.npy')

In [5]:
class Custom_lr(keras.callbacks.Callback):

    def on_train_begin(self, logs={}):
        K.set_value(self.model.optimizer.lr, 0.001)
 
    def on_epoch_begin(self, epoch, logs={}):
        lr_present=K.get_value(self.model.optimizer.lr)
        #print(epoch)
        if (epoch%10==0) and epoch:
            K.set_value(self.model.optimizer.lr, lr_present/((epoch)**0.5))
            print(K.get_value(self.model.optimizer.lr))
            print(lr_present/((epoch)**0.5))

VGG16 bottleneck

In [5]:
model = applications.VGG16(weights='imagenet', include_top=False)


x_train_bottleneck = model.predict(x_train)
x_cv_bottleneck = model.predict(x_cv)

np.save('dataset/x_train_bottleneck.npy', x_train_bottleneck)
np.save('dataset/x_cv_bottleneck.npy', x_cv_bottleneck)

print(x_train_bottleneck[0].shape)

(2, 2, 512)


In [6]:
x_train_bottleneck[0]

array([[[0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 1.2254674,
         0.       ]],

       [[0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ]]], dtype=float32)

In [7]:
x_train_bottleneck = np.load('dataset/x_train_bottleneck.npy')
x_cv_bottleneck = np.load('dataset/x_cv_bottleneck.npy')

In [8]:
x_train_bottleneck.shape[1:]

(2, 2, 512)

In [6]:
top_model=Sequential()
top_model.add(Flatten(input_shape=x_train_bottleneck.shape[1:]))
top_model.add(Dense(64, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

top_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [7]:
top_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                131136    
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 131,201
Trainable params: 131,201
Non-trainable params: 0
_________________________________________________________________


In [8]:
callbacks = [
    Custom_lr()
    ]

In [9]:
top_model.fit(x_train_bottleneck, y_train,
          epochs=35,
          batch_size=32,
          validation_data=(x_cv_bottleneck, y_cv), callbacks=callbacks,)

Epoch 1/35
213/213 [==============================] - 0s 2ms/step - loss: 1.4072 - accuracy: 0.5875 - val_loss: 0.6332 - val_accuracy: 0.6786
Epoch 2/35
213/213 [==============================] - 0s 1ms/step - loss: 0.6074 - accuracy: 0.6659 - val_loss: 0.6189 - val_accuracy: 0.6882
Epoch 3/35
213/213 [==============================] - 0s 1ms/step - loss: 0.5707 - accuracy: 0.6938 - val_loss: 0.6074 - val_accuracy: 0.6967
Epoch 4/35
213/213 [==============================] - 0s 1ms/step - loss: 0.5358 - accuracy: 0.7159 - val_loss: 0.6281 - val_accuracy: 0.6967
Epoch 5/35
213/213 [==============================] - 0s 1ms/step - loss: 0.5183 - accuracy: 0.7310 - val_loss: 0.6097 - val_accuracy: 0.7012
Epoch 6/35
213/213 [==============================] - 0s 1ms/step - loss: 0.4839 - accuracy: 0.7443 - val_loss: 0.6149 - val_accuracy: 0.7053
Epoch 7/35
213/213 [==============================] - 0s 1ms/step - loss: 0.4684 - accuracy: 0.7419 - val_loss: 0.6241 - val_accuracy: 0.7094
Epoch 

In [13]:
top_model.save('top_model_full_data_custom_lr_weights.h5') # best

In [6]:
# load VGG16
vgg_model=applications.VGG16(weights='imagenet', include_top=False, input_shape=(64,64,3))

In [7]:
model_aug=Sequential()
model_aug.add(vgg_model)

In [8]:
top_model=Sequential()
top_model.add(Flatten(input_shape=(2, 2, 512)))
top_model.add(Dense(64, activation='relu'))
top_model.add(Dense(1, activation='sigmoid'))
# top_model.load_weights('top_model_full_data_custom_lr_weights.h5')
model_aug.add(top_model)

In [9]:
for layer in model_aug.layers[0].layers[:17]:
    layer.trainable=False

In [10]:
model_aug.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(lr=1e-6), metrics=['accuracy'])

In [11]:
model_aug.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 2, 2, 512)         14714688  
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 131201    
Total params: 14,845,889
Trainable params: 2,491,009
Non-trainable params: 12,354,880
_________________________________________________________________


In [12]:
x_train[:10]

array([[[[ 93,  69,  48],
         [ 97,  71,  63],
         [102,  80,  65],
         ...,
         [ 94,  66,  45],
         [ 90,  68,  52],
         [ 90,  72,  53]],

        [[ 91,  70,  50],
         [ 98,  77,  61],
         [ 98,  75,  57],
         ...,
         [100,  75,  48],
         [ 96,  74,  53],
         [ 94,  69,  56]],

        [[ 95,  75,  58],
         [100,  84,  63],
         [ 96,  69,  50],
         ...,
         [102,  81,  52],
         [101,  75,  55],
         [ 98,  72,  58]],

        ...,

        [[ 87,  66,  46],
         [ 91,  70,  52],
         [ 96,  73,  51],
         ...,
         [ 98,  74,  56],
         [ 99,  71,  48],
         [ 93,  66,  46]],

        [[ 87,  66,  45],
         [ 92,  73,  54],
         [ 93,  71,  52],
         ...,
         [105,  80,  68],
         [109,  80,  65],
         [105,  78,  59]],

        [[ 86,  62,  39],
         [ 98,  76,  60],
         [ 94,  74,  58],
         ...,
         [107,  81,  66],
        

In [13]:
y_train[:10]

array([1, 0, 0, 0, 0, 0, 1, 1, 1, 0])

In [14]:
model_aug.fit(x_train, y_train, epochs=35, batch_size=32, validation_data=(x_cv, y_cv), verbose=1)

Epoch 1/35
213/213 [==============================] - 53s 247ms/step - loss: 3.6210 - accuracy: 0.5378 - val_loss: 3.1442 - val_accuracy: 0.5444
Epoch 2/35
213/213 [==============================] - 58s 272ms/step - loss: 3.0079 - accuracy: 0.5531 - val_loss: 2.8566 - val_accuracy: 0.5571
Epoch 3/35
213/213 [==============================] - 61s 288ms/step - loss: 2.6673 - accuracy: 0.5671 - val_loss: 2.6748 - val_accuracy: 0.5691
Epoch 4/35
213/213 [==============================] - 57s 267ms/step - loss: 2.4039 - accuracy: 0.5844 - val_loss: 2.5340 - val_accuracy: 0.5760
Epoch 5/35
213/213 [==============================] - 58s 274ms/step - loss: 2.1880 - accuracy: 0.6009 - val_loss: 2.4211 - val_accuracy: 0.5825
Epoch 6/35
213/213 [==============================] - 62s 289ms/step - loss: 2.0053 - accuracy: 0.6140 - val_loss: 2.3265 - val_accuracy: 0.5846
Epoch 7/35
213/213 [==============================] - 60s 283ms/step - loss: 1.8470 - accuracy: 0.6274 - val_loss: 2.2493 - val_ac

In [27]:
model_aug.predict(x_train[:20])

array([[0.5264753 ],
       [0.94470495],
       [0.66361344],
       [0.00169832],
       [0.02029553],
       [0.00563926],
       [0.942483  ],
       [0.9331857 ],
       [0.00266075],
       [0.5745946 ],
       [0.02291301],
       [0.9829025 ],
       [0.08444613],
       [0.9674344 ],
       [0.5789391 ],
       [0.9999796 ],
       [0.04543281],
       [0.9433596 ],
       [0.99575675],
       [0.07195842]], dtype=float32)

In [33]:
res = model_aug.predict(x_train[:4])


In [34]:
res

array([[0.5264753 ],
       [0.9447048 ],
       [0.66361344],
       [0.00169832]], dtype=float32)

In [35]:
y_train[:4]

array([1, 0, 0, 0])

In [15]:
model_aug.evaluate(x_train, y_train)

213/213 [==============================] - 34s 159ms/step - loss: 0.3364 - accuracy: 0.8696


[0.3364208936691284, 0.8695588111877441]

In [30]:
len(model_aug.predict(x_test[1]))

768

In [18]:
#fake images
allFakeResults=[]
for i in range(10):
    result = model_aug.predict(x_test[i])
    allFakeResults.append(max(result))

In [19]:
allFakeResults

[array([1.], dtype=float32),
 array([1.], dtype=float32),
 array([1.], dtype=float32),
 array([1.], dtype=float32),
 array([1.], dtype=float32),
 array([0.99983954], dtype=float32),
 array([0.99999666], dtype=float32),
 array([1.], dtype=float32),
 array([1.], dtype=float32),
 array([1.], dtype=float32)]

In [20]:
#pristine images
allPristineResults=[]
for i in range(10, 20):
    result = model_aug.predict(x_test[i])
    allPristineResults.append(max(result))

In [22]:
allPristineResults

[array([0.9999485], dtype=float32),
 array([1.], dtype=float32),
 array([1.], dtype=float32),
 array([1.], dtype=float32),
 array([1.], dtype=float32),
 array([1.], dtype=float32),
 array([1.], dtype=float32),
 array([1.], dtype=float32),
 array([1.], dtype=float32),
 array([1.], dtype=float32)]

In [26]:
x_test[0].shape

(682, 64, 64, 3)

In [27]:
x_train[0].shape

(64, 64, 3)

In [32]:
#pristine images dari train data deh :"D
allPristineResults=[]
for i in range(0,100,10):
    result = model_aug.predict(x_train[i:i+10])
    allPristineResults.append(max(result))

In [33]:
allPristineResults

[array([0.9959935], dtype=float32),
 array([0.99995804], dtype=float32),
 array([1.], dtype=float32),
 array([0.9818442], dtype=float32),
 array([0.99775153], dtype=float32),
 array([0.9900132], dtype=float32),
 array([0.9999999], dtype=float32),
 array([1.], dtype=float32),
 array([1.], dtype=float32),
 array([0.9767355], dtype=float32)]

In [35]:
#fake images
allFakeResults=[]
for i in range(5000,5100,10):
    result = model_aug.predict(x_train[i:i+10])
    allFakeResults.append(max(result))

In [36]:
allFakeResults

[array([0.9999933], dtype=float32),
 array([0.99972486], dtype=float32),
 array([1.], dtype=float32),
 array([0.999983], dtype=float32),
 array([0.9696905], dtype=float32),
 array([0.9976498], dtype=float32),
 array([0.99992055], dtype=float32),
 array([0.99961364], dtype=float32),
 array([0.9918443], dtype=float32),
 array([0.99995595], dtype=float32)]

In [16]:
model_aug.save('fine_tuned_model_adam_weights.h5')